In [2]:
# pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.legacy.data import Field, TabularDataset, Iterator

In [3]:
import sentencepiece as spm
import matplotlib.pyplot as plt

In [7]:
if torch.cuda.is_available():
    print('cuda')
    device = torch.device('cuda:0')
else:
    print('cpu')
    device = torch.device('cpu')

cuda


In [4]:
# Load sentence piece model

sp = spm.SentencePieceProcessor()
vocab_file = "./sp.model"
sp.load(vocab_file)

True

In [5]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(16001, 300)
        self.lstm1 = nn.LSTM(300, 128)
        self.lstm2 = nn.LSTM(128, 128)
        self.fc1 = nn.Linear(3840, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.embedding(x)
        x,_ = self.lstm1(x)
        x,_ = self.lstm2(x)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
model = Network()
model.load_state_dict(torch.load('./discriminator.model'))
model.eval()

Network(
  (embedding): Embedding(16001, 300)
  (lstm1): LSTM(300, 128)
  (lstm2): LSTM(128, 128)
  (fc1): Linear(in_features=3840, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
)

In [14]:
def process_sents(sents):
    sents = [sp.encode_as_ids(sent.strip()) for sent in sents]
    data_set = []
    for sent in sents:
        if len(sent) <= 30:
            sent.extend([0 for _ in range(30 - len(sent))])
        else:
            sent = sent[:30]
        data_set.append(sent)
    data_set = torch.tensor(data_set)
    return data_set

In [19]:
def classify_field(sents, sp, model):
    sents = process_sents(sents)
    
    fields = [0, 0, 0]
    for idx in range(len(sents)):
        sent = sents[idx].unsqueeze(0).long().to(device)
        output = model(sent)
        fields[torch.max(output.data, dim=1)[1].item()] += 1
    
    
    return ['movie', 'news', 'shopping'][fields.index(max(fields))]

In [20]:
# https://extmovie.com/movietalk/65867835
model.to(device)
text = [
    "넘버링이 길어질수록 초심을 잃고 방황하거나 기존의 장점을 잃고 퇴색해버리는 시리즈들의 예는 정말 셀 수가 없을 정도죠.",
    "특히 공포영화 시리즈라면 더더욱요.",
    "컨저링 3: 악마가 시켰다 는 이런 저주를 영리하게 피했다고도 할 수 있고 예외없이 맞이했다고도 볼 수 있습니다.",
    "무슨 말이냐면 이번 3편의 경우 기존 전작들의 공포 노선과는 방향성이 뚜렷하게 차이난다는 것이죠.",
    "제임스 완이 메가폰을 잡았던 1 2편의 경우 장소, 소품의 기믹을 활용한 창의적인 공포가 주를 이뤘습니다.",
    "장소의 경우 1편의 깜깜하게 암전된 지하실에서의 숨바꼭질 박수, 소품의 경우 2편의 걸어오는 액자 같은 경우죠.",
    "3편은 요로나의 저주로 악명..이 높았던 마이클 차베즈가 메가폰을 잡았는데, 시리즈 기존의 창의적 점프스케어와는 다르게 일반 공포영화에서 선보이는 밋밋하다면 밋밋할 수 있는 점프스케어를 연출합니다.",
    "점프스케어의 타이밍도 어지간히 읽히는 편이고, 확실히 전편들보다 아이디어의 부족에서 오는 식상함도 강하게 느껴지기는 합니다.",
    "다만 말씀드렸듯이 이는 반대로 생각해보면 일반적인 공포영화 스타일의 회귀로 시리즈 피로감을 어떻게든 줄여보려 했다는 역발상에서 왔다는 느낌도 지울 수가 없더군요.",
    "관점에 따라 달리보이는 영화라고 결론짓고 싶습니다.",
    "장소와 소품의 기믹을 활용한 이전의 컨저링을 더 보고 싶어서 간 거라면 식상한 점프스케어를 남발하는 지루한 공포영화로, 그냥 순수하게 읽히더라도 놀라는 재미나 기존의 공포영화가 목메이셨던 분들에게는 괜찮은 영화로 다가올 거 같네요.",
    "개인적으로는 드라마도 헐리웃 공식을 따라가면서 잘 짜여졌다 보는 편이고 워렌 부부를 맡은 패트릭 윌슨과 베라 파미가의 연기가 여전히 출중한 편이기 때문에 후자의 느낌으로 재밌게 봤습니다.",
    "그 요로나의 저주 감독이라 걱정이 앞서긴 했는데 그냥저냥 괜찮았다 보네요.",
]
classify_field(text, sp, model)

'movie'

In [21]:
# https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=028&aid=0002547295
model.to(device)
text = [
    "문재인 대통령이 4일 취임 뒤 두번째로 국가정보원을 방문해 박지원 국정원장으로부터 ‘개혁성과’를 보고받고 “우리는 국정원의 정치적 중립성을 확실하게 보장하겠다는 약속을 지켰다”고 말했다.",
    "문 대통령은 이날 ‘국정원 개혁성과 보고회’에서 “나는 지난 2018년 7월 이곳에서 결코 국정원을 정치적으로 이용하지 않겠고, 정권에 충성할 것을 요구하지 않을 것이며 정치적 중립성을 확실하게 보장하겠다고 약속했다”며 “나도, 여러분도 그 약속을 지켰다”고 말했다고 청와대는 밝혔다.",
    "또한 “이제 국정원이 다시 과거로 돌아가는 일은 없을 것이다.",
    "개혁의 주체가 된 국정원 직원들의 헌신적인 노력과 정부의 강력한 의지가 이룬 소중한 결실이자 국정원 역사에 길이 남을 찬란한 이정표가 아닐 수 없다”고 했다.",
    "아울러 문 대통령은 “국정원은 국내정보조직의 해편을 단행하고 의혹 사건에 대한 진실 규명 작업을 진행하는 한편, 정보활동부터 예산 집행에 이르기까지 적법 절차에 따라 투명하게 업무를 수행하는 문화를 정착시켰다”면서 “마침내 지난해 12월, 국가정보원법 전면 개정 입법을 통해 개혁의 확고한 제도화를 달성했다”고 평가했다.",
    "국정원은 이날 업무보고를 통해 국정원법 개정으로 국내 정보 업무가 폐지되었고, 방첩·대테러·사이버·우주정보 등의 업무가 구체화되거나 새로 추가됨에 따라 조직 체계 전반을 재정비했다고 밝혔다.",
    "대공수사권 이전과 관련해서는 올해 경찰과 합동수사를 진행했고, 새로운 협업수사 모델을 시범 운영하는 등 2023년 말까지 완전한 수사권 이관을 차질 없이 추진하겠다고 했다.",
    "또 변호사 자격을 가진 준법지원관이 업무 전 과정에서 준법 여부를 확인하고 있고, 외부 인권보호관을 위촉하는 등 외부 통제도 강화했다고 보고했다.",
    "박지원 원장은 “국정원은 국민의 요구와 정부의 강력한 의지, 전 직원의 노력으로 정치와 완전히 절연하고 북한·해외 전문 정보기관으로 다시 태어나고 있다. ‘일 잘하는 국정원’ ‘미래로 가는 국정원‘이 되겠다”고 말했다고 청와대는 전했다.",
    "문 대통령은 업무보고를 받기 전 순직한 정보 요원들을 기리기 위해 국정원 청사에 설치된 ‘이름없는 별’ 조형물 앞에서 묵념했다.",
    "최근 이 별은 18개에서 19개로 늘었다.",
    "문 대통령은 “지난 2018년 제막한 ‘이름 없는 별’에 그 사이 별 하나가 더해진 것에 대해 가슴이 아프다.",
    "살아서도, 죽어서도 이름과 직책조차 남기지 않은 채, 오직 ‘국익을 위한 헌신’이라는 명예만을 남긴 이름 없는 별들의 헌신에 고개 숙여 감사드린다”고 말했다.",
]
classify_field(text, sp, model)

'news'